In [2]:
import requests
import pandas as pd
import sqlite3
import time as tm
from datetime import datetime, timedelta
db_path = r'D:\Rfiles\量化投资学习\database\high_freq.db'

# 库表结构和数据类型

In [4]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# 遍历表名并获取每个表的列信息
for table in tables:
    table_name = table[0]
    print("Table: ", table_name)

    cursor.execute("SELECT COUNT(*) FROM {}".format(table_name))
    row_count = cursor.fetchone()[0]
    print("Row count: ", row_count)

    cursor.execute("PRAGMA table_info({})".format(table_name))
    columns = cursor.fetchall()
    for column in columns:
        column_name = column[1]
        column_type = column[2]
        print("Column: {}, Type: {}".format(column_name, column_type))

    print("\n")
conn.close()


Table:  Popular_stock
Row count:  3187225
Column: time, Type: TEXT
Column: thscode, Type: TEXT
Column: open, Type: REAL
Column: high, Type: REAL
Column: low, Type: REAL
Column: close, Type: REAL
Column: avgPrice, Type: REAL
Column: volume, Type: REAL
Column: amount, Type: REAL
Column: change, Type: REAL
Column: changeRatio, Type: REAL
Column: turnoverRatio, Type: REAL
Column: sellVolume, Type: INTEGER
Column: buyVolume, Type: INTEGER
Column: BBI, Type: REAL
Column: MA, Type: REAL
Column: MACD, Type: REAL
Column: TRIX, Type: REAL
Column: RSI, Type: REAL
Column: ARBR, Type: REAL
Column: BOLL, Type: REAL
Column: VOSC, Type: REAL
Column: ZDZB, Type: REAL


Table:  limit_up_stock
Row count:  3543905
Column: time, Type: TEXT
Column: thscode, Type: TEXT
Column: open, Type: REAL
Column: high, Type: REAL
Column: low, Type: REAL
Column: close, Type: REAL
Column: avgPrice, Type: REAL
Column: volume, Type: REAL
Column: amount, Type: REAL
Column: change, Type: REAL
Column: changeRatio, Type: REAL
C

In [7]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT l.thscode, s.SECURITY_NAME, min(l.time) FROM zhangtinggu l join Security_name s on l.thscode = s.THSCODE group by 1,2")
result = cursor.fetchall()
conn.close()
df = pd.DataFrame(result, columns=['thscode','SECURITY_NAME','min'])
df

,thscode,SECURITY_NAME,min
0,000004.SZ,国华网安,2023-08-21 09:30
1,000010.SZ,美丽生态,2023-08-21 09:30
2,000023.SZ,ST深天,2023-08-21 09:30
3,000025.SZ,特力A,2023-08-21 09:30
4,000046.SZ,*ST泛海,2023-08-21 09:30
...,...,...,...
309,688356.SH,键凯科技,2023-08-21 09:30
310,688418.SH,震有科技,2023-08-21 09:30
311,688523.SH,航天环宇,2023-08-21 09:30
312,688629.SH,华丰科技,2023-08-21 09:30


In [14]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT l.*, s.SECURITY_NAME FROM zhangtinggu l join Security_name s on l.thscode = s.THSCODE where l.thscode like '002235%' and l.time like '2023-10-31%' order by l.time")
result = cursor.fetchall()
conn.close()
df = pd.DataFrame(result)
df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,2023-10-31 09:30,002235.SZ,7.71,7.71,7.71,7.71,7.710000,604000.0,4656840.0,-0.12,...,7.805833,7.806,-0.009051,-0.005652,9.564560,107.692308,7.769130,27.462074,2.325036,安妮股份
1,2023-10-31 09:30,002235.SZ,7.71,7.71,7.71,7.71,7.710000,604000.0,4656840.0,-0.12,...,7.668854,7.676,0.012547,0.018906,91.059752,82.352941,7.621944,28.043149,2.342817,安妮股份
2,2023-10-31 09:31,002235.SZ,7.74,7.85,7.72,7.75,7.766012,1766100.0,13715553.0,0.04,...,7.794062,7.790,-0.012827,-0.010817,32.604766,160.000000,7.760314,31.520424,2.342817,安妮股份
3,2023-10-31 09:31,002235.SZ,7.74,7.85,7.72,7.75,7.766012,1766100.0,13715553.0,0.04,...,7.682708,7.694,0.018130,0.024016,95.070565,138.888889,7.608920,34.110485,2.360597,安妮股份
4,2023-10-31 09:32,002235.SZ,7.75,7.82,7.75,7.81,7.785275,757900.0,5900460.0,0.06,...,7.791250,7.786,-0.010853,-0.013662,53.794112,214.285714,7.760077,33.351525,2.360597,安妮股份
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,2023-10-31 14:58,002235.SZ,7.67,7.67,7.67,7.67,7.670000,29800.0,228566.0,0.01,...,7.657917,7.662,0.008742,0.015474,72.231990,80.000000,7.629249,26.865669,2.272605,安妮股份
478,2023-10-31 14:59,002235.SZ,7.67,7.67,7.67,7.67,NaN,0.0,0.0,0.00,...,7.659375,7.664,0.009120,0.016033,72.231990,84.210526,7.629341,24.311123,2.307255,安妮股份
479,2023-10-31 14:59,002235.SZ,7.67,7.67,7.67,7.67,NaN,0.0,0.0,0.00,...,7.659375,7.664,0.009120,0.016033,72.231990,84.210526,7.629341,24.311123,2.307255,安妮股份
480,2023-10-31 15:00,002235.SZ,7.67,7.67,7.67,7.67,7.670000,600700.0,4607369.0,0.00,...,7.661250,7.666,0.009312,0.016571,72.231990,83.333333,7.628876,24.655420,2.325036,安妮股份
